## **API Portal da Transparência**

### **Cadastro para uso da API**
Para fazer uso da API, deve-se realizar o cadastro de um e-mail em:<br />
http://portaldatransparencia.gov.br/api-de-dados/cadastrar-email.<br />
Através do e-mail cadastrado, você receberá um token, que deverá ser usado nas suas consultas via API.

### **Referência para uso da API**
No link abaixo, visualização do swagger para o teste dos endpoints:<br />
https://api.portaldatransparencia.gov.br/swagger-ui/index.html

No link abaixo, referência de como fazer uma request para um endpoint da API:<br />
http://www.portaldatransparencia.gov.br/pagina-interna/603579-api-de-dados-exemplos-de-uso

**Abaixo, segue a URL base para todos os endpoints:**<br />
**http://api.portaldatransparencia.gov.br/**

### **Endpoint de Licitações**
**GET -> /api-de-dados/licitacoes | Descrição: Consulta todas as licitações do Poder Executivo Federal**<br /><br />
**Os filtros mínimos para esse endpoint são:**<br /><br />
**Página (padrão = 1); Período de no máximo 1 mês; Código do Órgão (SIAFI)**<br /><br />
**A seguir a lista de todos os campos que podem ser fornecidos, e seus padrões:**<br />
<pre>
1° - Campo: dataInicial | Tipo: string | Formato: (DD/MM/AAAA) | Descrição: Data de abertura inicial<br />
2° - Campo: dataFinal   | Tipo: string | Formato: (DD/MM/AAAA) | Descrição: Data de abertura final<br />
3° - Campo: codigoOrgao | Tipo: string | Formato: "1000"       | Descrição: Código do Órgão (SIAFI)<br />
4° - Campo: pagina      | Tipo: int32  | Formato: 1            | Descrição: Página consultada<br />
</pre>
**OBS: Os códigos dos Órgão no SIAFI podem ser obtidos no seguinte link:**<br />
https://www.tesourotransparente.gov.br/ckan/dataset/lista-de-orgaos-do-siafi<br />

### **Return Values**<br />
Esse endpoint tem os seguintes valores de retorno de acordo com o Status Code:<br />

**Status Code: 200 (OK)**<br />
**Return Type: JSON**<br />
**Return Values:**<br />
<div style="overflow-y: scroll; height: 200px; width: 600px; border: 1px solid #ccc; padding: 10px;">
<pre>
[
  {
    "id": 0,
    "licitacao": {
      "numero": "string",
      "objeto": "string",
      "numeroProcesso": "string",
      "contatoResponsavel": "string"
    },
    "dataResultadoCompra": "2024-06-07",
    "dataAbertura": "2024-06-07",
    "dataReferencia": "2024-06-07",
    "dataPublicacao": "2024-06-07",
    "situacaoCompra": "string",
    "modalidadeLicitacao": "string",
    "instrumentoLegal": "string",
    "valor": 0,
    "municipio": {
      "codigoIBGE": "string",
      "nomeIBGE": "string",
      "codigoRegiao": "string",
      "nomeRegiao": "string",
      "pais": "string",
      "uf": {
        "sigla": "string",
        "nome": "string"
      }
    },
    "unidadeGestora": {
      "codigo": "string",
      "nome": "string",
      "descricaoPoder": "string",
      "orgaoVinculado": {
        "codigoSIAFI": "string",
        "cnpj": "string",
        "sigla": "string",
        "nome": "string"
      },
      "orgaoMaximo": {
        "codigo": "string",
        "sigla": "string",
        "nome": "string"
      }
    }
  }
]
</pre>
</div>

**Status Code: 400 (Bad Request)**<br />
**Return Type: JSON**<br />
**Return Values: {}**<br />

**Status Code: 401 (Unauthorized)**<br />
**Return Type: JSON**<br />
**Return Values: {}**<br />

**Status Code: 500 (Internal Server Error)**<br />
**Return Type: JSON**<br />
**Return Values: {}**<br />

In [3]:
# Reading list of SIAFI codes from CSV file
import pandas as pd
import os

file_name = "lista-de-orgaos-do-siafiListadeOrgaosSIAFIListadeOrgaosSIAFI2024-05-0104.23.24.506.csv"
file_path = os.getcwd() + "\\data\\lista-orgaos-siafi\\" + file_name

df = pd.read_csv(file_path)
df

,Órgão UGE Código,Órgão UGE Nome,Órgão UGE CNPJ,Órgão UGE - Poder Código,Órgão UGE - Poder Nome,Órgão UGE - Tipo Administração Código,Órgão UGE - Tipo Administração Nome
0,1000,CAMARA DOS DEPUTADOS,530352000159,1,PODER LEGISLATIVO,1,ADMINISTRACAO DIRETA
1,1901,FUNDO ROTATIVO DA CAMARA DOS DEPUTADOS,26994574000116,1,PODER LEGISLATIVO,7,FUNDOS
2,2000,SENADO FEDERAL,530279000115,1,PODER LEGISLATIVO,1,ADMINISTRACAO DIRETA
3,2001,FUNDO ESPECIAL DO SENADO FEDERAL,530279000115,1,PODER LEGISLATIVO,7,FUNDOS
4,2002,SENADO FEDERAL/FUNSEEP,530279000115,1,PODER LEGISLATIVO,7,FUNDOS
...,...,...,...,...,...,...,...
580,97120,ESTADO DO MATO GROSSO DO SUL,0,0,PODER EXECUTIVO,12,ADMINISTRACAO DIRETA MUNICIPAL
581,97220,ESTADO DO MATO GROSSO,0,0,PODER EXECUTIVO,12,ADMINISTRACAO DIRETA MUNICIPAL
582,97320,ESTADO DE GOIAS,0,0,PODER EXECUTIVO,12,ADMINISTRACAO DIRETA MUNICIPAL
583,97400,DISTRITO FEDERAL,0,0,PODER EXECUTIVO,11,ADMINISTRACAO DIRETA ESTADUAL


In [34]:
# Remebmer "pip install ipywidgets" to use progressbar on notebook
# Retrieving information from API from all codes from a specified date period until we find something that is not NULL
import requests
import time
from tqdm.notebook import tqdm
from api_key import API_KEY

DEFAULT_SLEEP_TIME = 0.5 # In Seconds
headers = {"chave-api-dados":API_KEY}
data_inicial = "01%2F05%2F2024"
data_final = "31%2F05%2F2024"
pagina = 1

json_data_lst = []

lista_codigos_siafi = df["Órgão UGE Código"].to_list()
for codigo_orgao in tqdm(lista_codigos_siafi, desc="Loading..."):
    url = f"http://api.portaldatransparencia.gov.br/api-de-dados/licitacoes?dataInicial={data_inicial}&dataFinal={data_final}&codigoOrgao={codigo_orgao}&pagina={pagina}"
    response = requests.get(url, headers=headers)
    if (response.text != "[]") or (response.status_code != 200):
        nome_orgao = df[df["Órgão UGE Código"]==codigo_orgao]["Órgão UGE Nome"].iloc[0]
        print(f"Response Status Code: {response.status_code}")
        print(f"Codigo Órgão SIAFI: {codigo_orgao}")
        print(f"Nome Órgão: {nome_orgao}\n")
        print(f"Response Text: {response.text}")
        json_data_lst.append(response.json()[0])
    time.sleep(DEFAULT_SLEEP_TIME)

Loading...:   0%|          | 0/585 [00:00<?, ?it/s]

Response Status Code: 200
Codigo Órgão SIAFI: 20502
Nome Órgão: FINANCIADORA DE ESTUDOS E PROJETOS

Response Text: [{"id":912491542,"licitacao":{"numero":"900052024","objeto":"Objeto: Pregão Eletrônico -  Prestação de serviços de cotação de preços, reserva, emissão, cancelamento, remarcação e fornecimento de passagens aéreas nacionais e internacionais e rodoviárias (terrestre), seguro-viagem, locação de transportes e serviços correlatos no Brasil e no exterior","numeroProcesso":"FP-ADM-2023/02308","contatoResponsavel":""},"dataResultadoCompra":null,"dataAbertura":"2024-05-02","dataReferencia":"2024-04-09","dataPublicacao":"1900-01-01","situacaoCompra":"A Divulgar","modalidadeLicitacao":"Pregão","instrumentoLegal":"Sem informação","valor":0.0000,"municipio":{"codigoIBGE":"3304557","nomeIBGE":"RIO DE JANEIRO","codigoRegiao":"3","nomeRegiao":"SUDESTE","pais":"BRASIL","uf":{"sigla":"RIO DE JANEIRO","nome":"RJ"}},"unidadeGestora":{"codigo":"365001","nome":"FINANCIADORA DE ESTUDOS E PROJETOS

In [35]:
## Exporting JSON response to file
import json
import os

json_response_file_path = os.getcwd() + "\\json_responses\\"
json_file_name = "portal-da-transparencia-api-result.json"

if not os.path.exists(json_response_file_path):
    os.mkdir(json_response_file_path)
    print(f"Created path: {json_response_file_path}")

with open(json_response_file_path+json_file_name, "w+", encoding="utf-8") as f:
    json.dump(json_data_lst, f)

In [37]:
## Flatening Json so we can easily turn it into a DataFrame and perform analysis
def flatten_json(json, parent_key='', sep='_'):
    """
    Flatten a nested json object.
    """
    items = []
    for key, value in json.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_json(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))
    return dict(items)

## Turning each json object from before into a flattened version, creating a dataframe and inserting it into a df list
df_json_lst = []
for json_obj in json_data_lst:
    flat_json = flatten_json(json_obj)
    df_responses = pd.json_normalize(flat_json)
    df_json_lst.append(df_responses)

## If we have a df list with contents, concat them into a final df
df_final = []
if df_json_lst:
    df_final = pd.concat(df_json_lst).reset_index(drop=True)

df_final

,id,licitacao_numero,licitacao_objeto,licitacao_numeroProcesso,licitacao_contatoResponsavel,dataResultadoCompra,dataAbertura,dataReferencia,dataPublicacao,situacaoCompra,...,unidadeGestora_codigo,unidadeGestora_nome,unidadeGestora_descricaoPoder,unidadeGestora_orgaoVinculado_codigoSIAFI,unidadeGestora_orgaoVinculado_cnpj,unidadeGestora_orgaoVinculado_sigla,unidadeGestora_orgaoVinculado_nome,unidadeGestora_orgaoMaximo_codigo,unidadeGestora_orgaoMaximo_sigla,unidadeGestora_orgaoMaximo_nome
0,912491542,900052024,Objeto: Pregão Eletrônico - Prestação de serv...,FP-ADM-2023/02308,,None,2024-05-02,2024-04-09,1900-01-01,A Divulgar,...,365001,FINANCIADORA DE ESTUDOS E PROJETOS,EXECUTIVO,20502,33749086000613,,Financiadora de Estudos e Projetos,24000,,"Ministério da Ciência, Tecnologia e Inovação"
1,914426519,900012024,Objeto: Pregão Eletrônico - A presente tem po...,11080000152202461,,None,2024-05-07,2024-04-22,2024-04-22,A Publicar,...,170175,SUPERINT.REG. ADM. DO MGI - RIO GRANDE SUL,EXECUTIVO,46000,00489828000155,,Ministério da Gestão e da Inovação em Serviços...,46000,,Ministério da Gestão e da Inovação em Serviços...
2,911196243,900012024,Objeto: Pregão Eletrônico - Pregão 01/2024 - ...,64615006052202420,,None,2024-05-02,2024-04-05,2024-04-05,Publicado,...,160501,MUSEU HISTORICO DO EXERCITO FORTE COPACABANA,EXECUTIVO,52121,00394452000103,,Comando do Exército,52000,,Ministério da Defesa
3,913634694,900062024,Objeto: Pregão Eletrônico - Constitui objeto ...,2024-PR-0006-FE,,None,2024-05-13,2024-04-04,1900-01-01,Divulgado,...,168008,INDUSTRIA DE MATERIAL BELICO DO BRASIL/FE,EXECUTIVO,52221,00444232000139,,Indústria de Material Bélico do Brasil,52000,,Ministério da Defesa
4,910431087,002962023,Objeto: Pregão Eletrônico - Prestação de serv...,DAN.A/PE-296/2023,,None,2024-05-03,2024-04-08,1900-01-01,A Divulgar,...,910847,ELETROBRAS TERMONUCLEAR S/A,EXECUTIVO,91081,00000000000000,,Empresas de Energia,32000,,Ministério de Minas e Energia
